# Experiment 14

In [ ]:
%tensorflow_version 2.x
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
colab = 1
FOLDER = 'Experiment_14'

In [ ]:
!pip install -q gcsfs
import os
import re, gc
import numpy as np
import pandas as pd
import random
import math
from sklearn import metrics
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K
from tqdm.notebook import tqdm as tqdm
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional

     |████████████████████████████████| 92kB 4.8MB/s 
     |████████████████████████████████| 1.2MB 11.5MB/s 
     |████████████████████████████████| 266kB 24.6MB/s 
     |████████████████████████████████| 153kB 38.2MB/s 


In [ ]:
PATH = '/content/drive/My Drive/Kaggle/rsna_pulmonary'
PATH = os.path.join(PATH, FOLDER)
os.makedirs(PATH, exist_ok=True)
PATH

'/content/drive/My Drive/Kaggle/rsna_pulmonary/Experiment_14'

In [ ]:
GCS_PATHS = [
'gs://kds-675a4d0ca17253f876424309f74145c1167e831dc34c32522a29e4a9',
'gs://kds-1077ea6ef5bd8d0ba8150b786cee31c1c897708727eeb60a5fc2c0cd',
'gs://kds-1bc7664eb4ada6b80cb8c57e621c6cb78f2092fb164aea1302bd77a3',
'gs://kds-19a534b62267f9fe27cf1640e8781e6c04fd661d6b67ac160f1cc746',
'gs://kds-b5a1644b1f9084284cb2f724b41a5501d7fa4c9e5c4eb99a5a8d602f',
'gs://kds-f928be364facd6fe31db68406c543692388b880e8c7585f4c0403573',
'gs://kds-1a521e2e4a90df381780b48a4ce28e63764e7bec29805c8d6d6861e2',
'gs://kds-407358bc70721c44703d11afe83bdf2e970449eb110f376d01223862',
'gs://kds-db1695bd356de02fb8b89de44067a54e1db12f99675678d5e92b5917',
'gs://kds-2d74bd2971f4de2c28e3a427d05381f967de5070458b20031273e0f8',
'gs://kds-80c22cf93ec97373e7bbdc091dc2f9f7358e4c08d85797cdf86d9723',
'gs://kds-981f9b6ce021f08600630510b5398dafaca9ef15de304a20f083c886',
'gs://kds-54fafeca23b3db6855d075ce547486a53f37f981e0100552e714f561',
'gs://kds-9c42ec420192f5d63b360164ad37ed81dbfe7afc16a4e4fac418e952',
'gs://kds-1bc45eb2a6dcbb73ace5842a7b5c01effd9a3230f9b8c73314a26d74'
]

DICT_PATH = 'gs://kds-baad38a91ebb50744b66774d8ba77e8d3df2ce699dc5f8ad2d5dc6f6/train_with_metadata.csv'

In [ ]:
SEED = 100

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

FILENAMES=[]
for GCS_PATH in GCS_PATHS:     
    FILENAMES += tf.io.gfile.glob(GCS_PATH + '/train*.tfrec')
TRAINING_FILENAMES, VALIDATION_FILENAMES = train_test_split(FILENAMES, test_size = 0.02, random_state = SEED)
training_groups = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in TRAINING_FILENAMES]
validation_groups = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in VALIDATION_FILENAMES]
print(training_groups, validation_groups)

df = pd.read_csv(DICT_PATH)
def count_data_items(filenames):
    records = [int(re.compile(r"_([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    n = df[df['fold'].isin(records)].shape[0]
    return n

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES  = count_data_items(VALIDATION_FILENAMES)
print(f'Training with {NUM_TRAINING_IMAGES} images')
print(f'Validating with {NUM_VALIDATION_IMAGES} images')

[1, 12, 13, 6, 10, 5, 2, 4, 0, 11, 7, 3, 14, 8] [9]
Training with 1671055 images
Validating with 119539 images


# TRAIN LSTM

In [ ]:
seed_everything(SEED)

# train
df_train = pd.read_csv(DICT_PATH)
df_train = df_train[df_train.fold!=9]
df_train = df_train.sort_values(by=['StudyInstanceUID','image_position'])

import pickle
with open(os.path.join(PATH, f'mapping_train_set.p'), 'rb') as fp:
    dict_ = pickle.load(fp)

print(len(dict_),len(df_train))

df_studywise = df_train.groupby('StudyInstanceUID').mean()
df_studywise['count'] = df_train.groupby('StudyInstanceUID')['SOPInstanceUID'].count()
df_studywise['SOPInstanceUID'] = df_train.groupby('StudyInstanceUID')['SOPInstanceUID'].unique()
df_studywise['SOPInstanceUID'] = df_studywise['SOPInstanceUID'].apply(lambda x:list(x))
df_studywise['predictions'] = df_studywise['SOPInstanceUID'].apply(lambda x:np.stack([dict_[i] for i in x], axis=0))

print(df_studywise.shape)

del dict_
gc.collect()

1671055 1671055
(6794, 24)


In [ ]:
# val
df_val = pd.read_csv(DICT_PATH)
df_val = df_val[df_val.fold==9]
df_val = df_val.sort_values(by=['StudyInstanceUID','image_position'])

import pickle
with open(os.path.join(PATH, f'mapping_validation_set.p'), 'rb') as fp:
    dict_val = pickle.load(fp)

print(len(dict_val),len(df_val))

df_studywise_val = df_val.groupby('StudyInstanceUID').mean()
df_studywise_val['count'] = df_val.groupby('StudyInstanceUID')['SOPInstanceUID'].count()
df_studywise_val['SOPInstanceUID'] = df_val.groupby('StudyInstanceUID')['SOPInstanceUID'].unique()
df_studywise_val['SOPInstanceUID'] = df_studywise_val['SOPInstanceUID'].apply(lambda x:list(x))
df_studywise_val['predictions'] = df_studywise_val['SOPInstanceUID'].apply(lambda x:np.stack([dict_val[i] for i in x], axis=0))

print(df_studywise_val.shape)
print(df_studywise_val['predictions'][0].shape)

del dict_val
gc.collect()

119539 119539
(485, 24)
(230, 128)


46

In [ ]:
temp_train = df_studywise[df_studywise['count']<=300]
temp_train.shape

(6138, 24)

In [ ]:
def exam_level_loss(y_true,y_pred):
    epsilon = 1e-7
    y_pred = tf.clip_by_value(y_pred,epsilon,1-epsilon)
    weights = tf.constant([ 0.0736196319,
                            0.09202453988, 
                            0.1042944785, 
                            0.1042944785, 
                            0.1877300613, 
                            0.06257668712, 
                            0.06257668712,
                            0.2346625767,
                            0.0782208589],dtype=tf.float32)
    loss = -(y_true*tf.math.log(y_pred) + (1-y_true)*tf.math.log(1-y_pred))
    loss = tf.reduce_sum(loss*weights, axis=1)
    loss = tf.reduce_mean(loss, axis=0)
    return loss

In [ ]:
RETRAIN = True
EPOCHS = 10
MODEL_PATH = 'bilstm_epoch-10_loss-0.3208_valloss-0.2900.h5'
INITIAL_EPOCH = 0
EMBEDDING_DIM = 128
BATCH_SIZE = 50
LR = 1e-4

model = Sequential()
model.add(Bidirectional(LSTM(1024, return_sequences=True), input_shape=(None,EMBEDDING_DIM)))
model.add(Bidirectional(LSTM(1024, return_sequences=False)))
model.add(Dense(9, activation='sigmoid'))

#opt = tf.keras.optimizers.Adam(learning_rate = 1e-6)
opt = tf.keras.optimizers.SGD(learning_rate = LR, momentum=0.99)
model.compile(loss=exam_level_loss, 
              optimizer=opt,
              metrics = ['AUC'])
if RETRAIN:
    print('Loading model...')
    model.load_weights(os.path.join(PATH, MODEL_PATH))

ordered_columns = ['negative_exam_for_pe','indeterminate','chronic_pe','acute_and_chronic_pe','central_pe','leftsided_pe','rightsided_pe','rv_lv_ratio_gte_1','rv_lv_ratio_lt_1' ]
#ordered_columns = ['leftsided_pe']

def train_generator():
    X = []
    Y = []
    while True:
        for index,row in temp_train.iterrows():
            y_train = tf.convert_to_tensor(row[ordered_columns].values.astype(np.float32))
            x_train = tf.convert_to_tensor(row['predictions'])
            length = x_train.shape[0]
            left = (300-length)//2
            right = 300-length-left
            x_train = tf.pad(x_train, [[left,right],[0,0]], constant_values=0)
            X.append(x_train)
            Y.append(y_train)
            if len(X)==BATCH_SIZE:
                x_batch = tf.stack(X, axis=0)
                y_batch = tf.stack(Y, axis=0)
                yield x_batch, y_batch
                del X, Y
                X = []
                Y = []

def val_generator():
    while True:
        for index,row in df_studywise_val.iterrows():
            x_val = tf.convert_to_tensor(row['predictions'])
            x_val = tf.expand_dims(x_val,axis=0)
            y_val = tf.convert_to_tensor(row[ordered_columns].values.astype(np.float32))
            y_val = tf.expand_dims(y_val,axis=0)
            yield x_val, y_val

checkpoint = tf.keras.callbacks.ModelCheckpoint(os.path.join(PATH, 'bilstm_epoch-{epoch}_loss-{loss:.4f}_valloss-{val_loss:.4f}.h5'), 
                                                 save_best_only = False, 
                                                 save_weights_only = False)

Loading model...


In [ ]:
model.fit(train_generator(), steps_per_epoch=len(temp_train)//BATCH_SIZE, 
          validation_data=val_generator(), validation_steps=len(df_studywise_val),
          callbacks=[checkpoint],epochs=EPOCHS, verbose=1, initial_epoch=INITIAL_EPOCH)

Epoch 1/10
122/122 [==============================] - 392s 3s/step - loss: 0.3215 - auc: 0.8202 - val_loss: 0.2899 - val_auc: 0.8416
Epoch 2/10
122/122 [==============================] - 389s 3s/step - loss: 0.3211 - auc: 0.8206 - val_loss: 0.2899 - val_auc: 0.8406
Epoch 3/10
122/122 [==============================] - 389s 3s/step - loss: 0.3201 - auc: 0.8213 - val_loss: 0.2898 - val_auc: 0.8409
Epoch 4/10
122/122 [==============================] - 390s 3s/step - loss: 0.3211 - auc: 0.8208 - val_loss: 0.2897 - val_auc: 0.8408
Epoch 5/10
122/122 [==============================] - 390s 3s/step - loss: 0.3207 - auc: 0.8211 - val_loss: 0.2898 - val_auc: 0.8406
Epoch 6/10
122/122 [==============================] - 386s 3s/step - loss: 0.3204 - auc: 0.8213 - val_loss: 0.2900 - val_auc: 0.8411
Epoch 7/10
122/122 [==============================] - 388s 3s/step - loss: 0.3204 - auc: 0.8214 - val_loss: 0.2903 - val_auc: 0.8405
Epoch 8/10
122/122 [==============================] - 388s 3s/step - 

# FINAL VALIDATION SCORE

In [ ]:
IMAGE_SCORE = 0.27
EXAM_SCORE = 0.23
def final_score(image_score, exam_score, q_i_sum, count):
    score = (count*exam_score)+(q_i_sum*0.07361963*image_score)
    score = score/((q_i_sum*0.07361963)+count)
    return score
temp_df = df[df.fold==9]
final_score(IMAGE_SCORE, EXAM_SCORE, temp_df['q_i'].sum(), temp_df['StudyInstanceUID'].nunique())

0.24865383161695528